In [ ]:
Komentarz:
    To możliwe, że Naive Bayes radził sobie lepiej :) 
        https://stackoverflow.com/questions/35360081/naive-bayes-vs-svm-for-classifying-text-data
            
Ogólnie - do przemyślenia, może da radę lepiej...

In [1]:
import data_helpers
import os
import pandas as pd

load_main = True

main_cats = ['ActionName', 'Capability']
dataframes = {cat : {} for cat in main_cats} if load_main else {}

for cat in main_cats:
    path = os.getcwd() + "/" + cat + "/Processed/"
    ls = os.listdir(path)
    for file in ls:
        if (load_main and cat in file) or (not load_main and cat not in file):
            df = pd.read_csv(path + file, encoding='utf-8')

            df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df.rename({"Unnamed: 0.1":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df_type = file.split('-')[1]
            if load_main:
                dataframes[cat][df_type] = df
            else:
                sub_cat = file.split('-')[0]
                if sub_cat not in dataframes.keys():
                    dataframes[sub_cat] = {}
                dataframes[sub_cat][df_type] = df

if not load_main:
    main_cats = list(dataframes.keys())
main_cats

['ActionName', 'Capability']

In [2]:
from clf_helpers import do_the_pipeline

def dummy(doc):
    return doc

In [66]:
SVC??

In [4]:
from nbsvm import NBSVM

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
import numpy as np
    
pipeline = Pipeline([      
    ('vectorizer',
#      TfidfVectorizer(
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 2),
         analyzer='word',
         binary=False,
#          max_features=2000
     )),
    ('clf', SVC(
        C=100.0, 
        kernel='rbf',
        gamma = 0.001, 
        class_weight = 'balanced'
    )),
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 98.3572%
              precision    recall  f1-score   support

  ActionName       0.96      0.99      0.98      1154
NoActionName       1.00      0.98      0.99      2194

   micro avg       0.98      0.98      0.98      3348
   macro avg       0.98      0.99      0.98      3348
weighted avg       0.98      0.98      0.98      3348

- Zbiór testowy:
	ACCURACY: 77.6371%
              precision    recall  f1-score   support

  ActionName       0.65      0.72      0.69        80
NoActionName       0.85      0.80      0.83       157

   micro avg       0.78      0.78      0.78       237
   macro avg       0.75      0.76      0.76       237
weighted avg       0.78      0.78      0.78       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 98.1481%
              precision    recall  f1-score   support

  Ca

In [46]:
'''
Grid Search here, for one of the classes
- still not very satisfactory ...
'''

from sklearn.model_selection import GridSearchCV

parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
        'C': [10, 50, 100, 200, 300]},
    {'kernel': ['linear'], 'C': [10, 50, 100, 200, 300]}
]

scores = ['roc_auc', 'accuracy', 'precision', 'recall']

vectorizer = CountVectorizer(
                 preprocessor=dummy, 
                 ngram_range=(1, 1),
                 analyzer='word'
             )

train = dataframes['ActionName']['Train']
test = dataframes['ActionName']['Test']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score.upper())
    print()

    clf = GridSearchCV(SVC(), parameters, cv=5,
#                        scoring='%s_macro' % score
                       scoring=score
                      )
    
    pipeline = Pipeline([      
        ('vectorizer', vectorizer),
        ('clf', clf ),
    ])

    pipeline.fit(train['text-rel-tokens'], train['label_num'])
    clf = pipeline.named_steps['clf']

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = test['label_num'], pipeline.predict(test['text-rel-tokens'])
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for ROC_AUC

Best parameters set found on development set:

{'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.734 (+/-0.119) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.716 (+/-0.119) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.759 (+/-0.110) for {'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}
0.719 (+/-0.124) for {'C': 50, 'gamma': 0.0001, 'kernel': 'rbf'}
0.757 (+/-0.092) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.734 (+/-0.119) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.754 (+/-0.063) for {'C': 200, 'gamma': 0.001, 'kernel': 'rbf'}
0.751 (+/-0.125) for {'C': 200, 'gamma': 0.0001, 'kernel': 'rbf'}
0.752 (+/-0.052) for {'C': 300, 'gamma': 0.001, 'kernel': 'rbf'}
0.754 (+/-0.123) for {'C': 300, 'gamma': 0.0001, 'kernel': 'rbf'}
0.727 (+/-0.022) for {'C': 10, 'kernel': 'linear'}
0.707 (+/-0.049) for {'C': 50, 'kernel': 'linear'}
0.698 (+/-0.045) for {'C': 100, 'kernel': 'linear'}
0.696 (+/-0.041) for {'C': 200, 

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Best parameters set found on development set:

{'C': 200, 'gamma': 0.0001, 'kernel': 'rbf'}

Grid scores on development set:

0.655 (+/-0.263) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.662 (+/-0.125) for {'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}
0.636 (+/-0.450) for {'C': 50, 'gamma': 0.0001, 'kernel': 'rbf'}
0.650 (+/-0.094) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.641 (+/-0.264) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.624 (+/-0.075) for {'C': 200, 'gamma': 0.001, 'kernel': 'rbf'}
0.681 (+/-0.143) for {'C': 200, 'gamma': 0.0001, 'kernel': 'rbf'}
0.612 (+/-0.069) for {'C': 300, 'gamma': 0.001, 'kernel': 'rbf'}
0.666 (+/-0.150) for {'C': 300, 'gamma': 0.0001, 'kernel': 'rbf'}
0.561 (+/-0.049) for {'C': 10, 'kernel': 'linear'}
0.531 (+/-0.059) for {'C': 50, 'kernel': 'linear'}
0.522 (+/-0.056) for {'C': 100, 'kernel': 'linear'}
0.519 (+/-0.058) for {'C': 200, 'kernel': 'linear'}
0.518 (+/-0.057) 

In [37]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
import numpy as np
    
pipeline = Pipeline([      
    ('vectorizer',
#      TfidfVectorizer(
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 2),
         analyzer='word',
         binary=False,
#          max_features=2000
     )),
    ('clf', NBSVM()),
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 67.1147%
              precision    recall  f1-score   support

  ActionName       0.98      0.05      0.09      1154
NoActionName       0.67      1.00      0.80      2194

   micro avg       0.67      0.67      0.67      3348
   macro avg       0.82      0.52      0.44      3348
weighted avg       0.77      0.67      0.55      3348

- Zbiór testowy:
	ACCURACY: 70.4641%
              precision    recall  f1-score   support

  ActionName       0.86      0.15      0.26        80
NoActionName       0.70      0.99      0.82       157

   micro avg       0.70      0.70      0.70       237
   macro avg       0.78      0.57      0.54       237
weighted avg       0.75      0.70      0.63       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 88.0227%
              precision    recall  f1-score   support

  Ca